# 아마존 세일즈 데이터 EDA - 보고서 작성용

## Introduction

> 앞선 3주 동안 약 **1400개** 행의 소규모 데이터를 분석하며, 기본적인 통계기법과 클러스터링에 대해 파악함.   
> 이번 EDA의 목적은 2023년의 인도, 영국, 미국, 캐나다의 **대규모** 마켓 세일즈 데이터(~150만개)를 분석함에 있음.

|#| Table of Contents | Finished |
|:--|:--:|:--:|
|1| Install Necessary Packages & Load Data | &check; |
|2| Acknowledge Characteristics of Data | &cross;|


&copy; 2024 Yoori Choi <it.glasschoi@gmail.com>
* * *

## 1. Install Necessary Packages & Load Data

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# Import Necessary Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import warnings

# from tqdm import tqdm_notebook
warnings.simplefilter(action='ignore')
%matplotlib inline

In [32]:
import sys
import os

# Get the full path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))

# Add the project root to sys.path
sys.path.insert(0, project_root)

# Now try to import
from functions.amazon_analysis import AmazonAnalyzer, AmazonDataframe

In [33]:
project_root

'/Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning'

In [34]:
analyzer = AmazonAnalyzer.from_config('config.json', project_root)

Attempting to access: /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon_india.csv
Loaded India from cache
Successfully added dataframe: India
Attempting to access: /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon_usa.csv
Loaded US from cache
Successfully added dataframe: US
Attempting to access: /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon_uk.csv
Loaded UK from cache
Successfully added dataframe: UK
Attempting to access: /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon_canada.csv
Loaded Canada from cache
Successfully added dataframe: Canada
Attempting to access: /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon.csv
Loaded India_2022 from cache
Successfully added dataframe: India_2022


In [35]:
if analyzer:
    print("Loaded dataframes:")
    for name, df_obj in analyzer.dataframes.items():
        try:
            print(f"- {name}: {df_obj.df.shape}")
        except Exception as e:
            print(f"- {name}: Error loading - {str(e)}")
    
    result = analyzer.compare_columns_presence()
    if result is not None:
        print(result)
    else:
        print("Failed to compare columns presence")
else:
    print("Failed to initialize analyzer from config.")

Loaded dataframes:
Successfully loaded CSV from /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon_india.csv
- India: (1497145, 14)
Successfully loaded CSV from /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon_usa.csv
- US: (1393614, 14)
Successfully loaded CSV from /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon_uk.csv
- UK: (2222724, 11)
Successfully loaded CSV from /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon_canada.csv
- Canada: (1988016, 14)
Successfully loaded CSV from /Users/yoorichoi/Documents/ds_study/Amazon_sales_machine_learning/data/amazon.csv
- India_2022: (1465, 16)
                      India     US     UK  Canada  India_2022
rating_count          False  False  False   False        True
product_id             True   True   True    True        True
review_id             False  False  False   False        True
img_link               True   True   True    T

In [36]:
india_df = analyzer.get_dataframe("India")
us_df = analyzer.get_dataframe("US")
uk_df = analyzer.get_dataframe("UK")
canada_df = analyzer.get_dataframe("Canada")
india_2022_df = analyzer.get_dataframe("India_2022")

In [37]:
# analyzer.clear_cache()

#### 나중에 prep_functions 파일이나 amazon_analysis로 옮길 수 있는 함수들

In [66]:
def format_korean_number(number):
    if number == 0:
        return "0원"
    
    units = ['원', '만', '억', '조']
    result = []
    
    # 음수 체크
    is_negative = number < 0
    number = abs(number)
    
    for i, unit in enumerate(units):
        unit_value = number % 10000
        if unit_value > 0:
            if i == 0:  # '원' 단위일 때는 쉼표 포함
                result.append(f"{unit_value:,}{unit}")
            else:
                result.append(f"{unit_value}{unit}")
        number //= 10000
        if number == 0:
            break
    
    # 결과를 역순으로 조합
    final_result = ' '.join(reversed(result))
    
    # 음수면 앞에 마이너스 표시
    if is_negative:
        final_result = f"-{final_result}"
        
    return final_result

In [114]:
# 결과를 예쁘게 포맷팅하여 출력
def print_top_items(results):
    if isinstance(results, pd.DataFrame):
        for idx, (name, price) in enumerate(zip(results[results.columns[0]], results[results.columns[1]]), 1):
            formatted_price = format_korean_number(price)  # 이전에 만든 함수 사용
            print(f"{idx}. {name}: {formatted_price}")
    else:
        for idx, (item, count) in enumerate(results.items(), 1):
            print(f"{idx}. {item}: {count}")

* * *
## 2. Acknowledge Characteristics of Data

In [38]:
# Compare columns across all dataframes
presence_df = analyzer.compare_columns_presence()
presence_df

,India,US,UK,Canada,India_2022
rating_count,False,False,False,False,True
product_id,True,True,True,True,True
review_id,False,False,False,False,True
img_link,True,True,True,True,True
product_name,True,True,True,True,True
product_link,True,True,True,True,True
discount_percentage,True,True,False,True,True
about_product,False,False,False,False,True
user_id,False,False,False,False,True
boughtInLastMonth,True,True,True,True,False


In [39]:
def get_common_columns_by_threshold(analyzer, min_true_count=4):
    """
    Get columns that are present (True) in at least min_true_count dataframes
    
    Parameters:
    analyzer (AmazonAnalyzer): The analyzer instance
    min_true_count (int): Minimum number of True values required (default=4)
    
    Returns:
    list: Column names that meet the threshold criteria
    """
    presence_df = analyzer.compare_columns_presence()
    if presence_df is None:
        return []
        
    # Sum True values across each row
    true_counts = presence_df.sum(axis=1)
    
    # Filter rows where count >= min_true_count
    # filtered_columns = presence_df[true_counts >= min_true_count].index.tolist()
    filtered_columns = presence_df[true_counts >= min_true_count]
    
    return filtered_columns

In [40]:
get_common_columns_by_threshold(analyzer, 5)

,India,US,UK,Canada,India_2022
product_id,True,True,True,True,True
img_link,True,True,True,True,True
product_name,True,True,True,True,True
product_link,True,True,True,True,True
discounted_price,True,True,True,True,True
category,True,True,True,True,True
rating,True,True,True,True,True


In [41]:
get_common_columns_by_threshold(analyzer, 4)

,India,US,UK,Canada,India_2022
product_id,True,True,True,True,True
img_link,True,True,True,True,True
product_name,True,True,True,True,True
product_link,True,True,True,True,True
discount_percentage,True,True,False,True,True
boughtInLastMonth,True,True,True,True,False
actual_price,True,True,False,True,True
discounted_price,True,True,True,True,True
discounted_price_KRW,True,True,True,True,False
category,True,True,True,True,True


* * *
### 국가 별 데이터 파악 - (1) 미국

In [57]:
us_df.head(3)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,reviews,img_link,product_link,isBestSeller,boughtInLastMonth,discounted_price_KRW,actual_price_KRW
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",Suitcases,139.99,0.00,0,4.5,0,https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,False,2000,190722,0
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,Suitcases,169.99,209.99,19,4.5,0,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,False,1000,231594,286089
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,Suitcases,365.49,429.99,15,4.6,0,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,False,300,497942,585817


In [92]:
category_analysis = analyzer.analyze_categories('US', top_n=20)


=== US 카테고리 분석 ===
총 카테고리 수: 248개
총 상품 수: 1,393,614개

상위 20개 카테고리:
 1. Girls' Clothing                          28,289개 (2.03%)
 2. Boys' Clothing                           24,165개 (1.73%)
 3. Toys & Games                             20,657개 (1.48%)
 4. Men's Shoes                              18,780개 (1.35%)
 5. Women's Handbags                         18,695개 (1.34%)
 6. Girls' Jewelry                           18,441개 (1.32%)
 7. Men's Clothing                           17,924개 (1.29%)
 8. Men's Accessories                        17,482개 (1.25%)
 9. Women's Clothing                         17,281개 (1.24%)
10. Women's Jewelry                          16,948개 (1.22%)
11. Travel Accessories                       16,099개 (1.16%)
12. Women's Shoes                            15,102개 (1.08%)
13. Home Storage & Organization              15,099개 (1.08%)
14. Women's Accessories                      13,845개 (0.99%)
15. Men's Watches                            12,427개 (0.89%)
16. Boys' Jewelry

In [90]:
analyzer.analyze_category_metrics('US', 'discounted_price_KRW')  # 카테고리별 평균 가격


=== 카테고리별 평균 discounted_price_KRW 분석 (상위 20개) ===
 1. Computer Servers                         212만 7,416원 (상품수: 281.0개)
 2. Computers & Tablets                      77만 9,109원 (상품수: 7,962.0개)
 3. Smart Home: Home Entertainment           54만 1,835원 (상품수: 154.0개)
 4. Smart Home: Vacuums and Mops             46만 9,971원 (상품수: 50.0개)
 5. Smart Home - Heating & Cooling           35만 6,582원 (상품수: 235.0개)
 6. Luggage Sets                             32만 3,651원 (상품수: 144.0개)
 7. Computer Monitors                        31만 5,491원 (상품수: 3,402.0개)
 8. Smart Home Thermostats - Compatibility Checker 24만 8,662원 (상품수: 18.0개)
 9. Computer External Components             23만 6,096원 (상품수: 2,366.0개)
10. Video Projectors                         23만 4,727원 (상품수: 787.0개)
11. Smart Home: Smart Locks and Entry        22만 7,556원 (상품수: 270.0개)
12. Camera & Photo                           22만 4,331원 (상품수: 6,016.0개)
13. Data Storage                             21만 9,734원 (상품수: 5,851.0개)
14. Smart Home: Lawn and

In [91]:
analyzer.analyze_category_metrics('US', 'rating') # 카테고리별 평균 평점


=== 카테고리별 평균 rating 분석 (상위 20개) ===
 1. Gift Cards                               4.83 (상품수: 139.0개)
 2. Health & Household                       4.57 (상품수: 714.0개)
 3. Industrial & Scientific                  4.55 (상품수: 4,412.0개)
 4. Household Supplies                       4.54 (상품수: 1,908.0개)
 5. Kitchen & Dining                         4.54 (상품수: 4,864.0개)
 6. Electrical Equipment                     4.52 (상품수: 7,820.0개)
 7. Food Service Equipment & Supplies        4.52 (상품수: 5,896.0개)
 8. Sports & Outdoors                        4.50 (상품수: 2,638.0개)
 9. Power Tools & Hand Tools                 4.50 (상품수: 3,797.0개)
10. Electronic Components                    4.49 (상품수: 1,751.0개)
11. Automotive Replacement Parts             4.48 (상품수: 8,267.0개)
12. Home Appliances                          4.48 (상품수: 6,945.0개)
13. Kids' Home Store                         4.47 (상품수: 1,861.0개)
14. Tools & Home Improvement                 4.46 (상품수: 1,705.0개)
15. Industrial Power & Hand Tools          

In [117]:
print(f"=== 할인 가격 기준 카테고리 순위 (상위 10개)  ===")

top_bestseller_cat = analyzer.top_items_multi_filter(
    df_name='US',
    column='category',
    sort_by='discounted_price_KRW',
    n=10
)

print_top_items(top_bestseller_cat)

=== 할인 가격 기준 카테고리 순위 (상위 10개)  ===
1. Data Storage: 2688만 2,574원
2. Fabric Decorating: 2643만 517원
3. Computer Servers: 2243만 6,920원
4. Computer Servers: 1720만 6,621원
5. Packaging & Shipping Supplies: 1705만 7,166원
6. Office Electronics: 1634만 6,049원
7. Data Storage: 1527만 8,092원
8. Computer Servers: 1479만 4,400원
9. Data Storage: 1378만 5,013원
10. PlayStation 3 Games, Consoles & Accessories: 1363만 7,602원


In [118]:
print(f"=== 할인 가격 기준 제품 순위 (상위 10개, 카테고리 무관)  ===")

top_bestseller = analyzer.top_items_multi_filter(
    df_name='US',                       # 데이터프레임 이름
    column='product_name',              # 보여줄 컬럼
    sort_by='discounted_price_KRW',     # 정렬 기준
    n=10                                # 결과 개수
)

print_top_items(top_bestseller)

=== 할인 가격 기준 제품 순위 (상위 10개, 카테고리 무관)  ===
1. Overland Storage Neoxl80 Storagelibrary Lto8 SAS: 2688만 2,574원
2. replicas Marinas: 2643만 517원
3. CISCO Systems 1 Port ATM Enhanced Oc12/Stm4 Single Mode Intermediate Reach: 2243만 6,920원
4. Cisco Systems 7140 Router Dual 10/100 Fe Dual Ac IP Sw: 1720만 6,621원
5. PARTNERS BRAND Corrugated Trash Can Plain - 40 Gallon: 1705만 7,166원
6. Sony VPL-XW6000ES 4K HDR Laser Home Theater Projector with Native 4K SXRD Panel, White: 1634만 6,049원
7. HPE MSA 2062 10GBASE-T iSCSI SFF Storage: 1527만 8,092원
8. CISCO Systems Catalyst 5000 Family Supervisor Iii with Enhanced Nffc Ii: 1479만 4,400원
9. Aruba a Hewlett Packard Enterprise company HPE 108 TB Hard Drive - 3.5" Internal - SAS (12Gb/s SAS): 1378만 5,013원
10. A-PS-SS72T-100, Black: 1363만 7,602원


In [120]:
print(f"=== 할인 가격 기준 제품 순위 (상위 10개, 카테고리: 'Suitcases')  ===")

# 가방 카테고리에서 할인가격이 가장 비싼 상품 10개
filters = {
    'category': 'Suitcases',
    # 'isBestSeller': True  # 필터 추가/삭제 가능
}

top_bestseller_suitcases = analyzer.top_items_multi_filter(
    df_name='US',                       # 데이터프레임 이름
    column='product_name',              # 보여줄 컬럼
    filters=filters,
    sort_by='discounted_price_KRW',     # 정렬 기준
    n=10                                # 결과 개수
)

print_top_items(top_bestseller_suitcases)

=== 할인 가격 기준 제품 순위 (상위 10개, 카테고리: 'Suitcases')  ===
1. Alpha 3 Worldwide Trip Expandable 4-Wheeled Packing Case - Large Suitcase with Top and Side-Grab Handles - Black: 203만 6,784원
2. Alpha 3 Extended Trip Expandable 4-Wheeled Packing Case Suitcase - Great for Extended Travel of Shared Packing - Rolling Luggage for Men and Women - Black: 190만 544원
3. Alpha 3 Medium Trip Expandable 4-Wheeled Packing Case - Large Suitcase with Top and Side-Grab Handles - Black: 176만 4,305원
4. Alpha Short Trip Expandable 4-Wheeled Packing Case - Roller Bag for Short Trips & Weekend Getaways - Carry-On Luggage with 4 Spinner Wheels - Travel Suitcase for Men & Women - Black: 162만 8,065원
5. Alpha 3 Short Trip Expandable 4 Wheeled Packing Case: 162만 8,065원
6. 30" International Trunk Aluminum Hardside Luggage: 153만 8,147원
7. Pursuit Aluminum Hardside Travel Case (Black, Continental Carry-On): 149만 1,825원
8. Pursuit Aluminum Hardside Travel Case (Bronze, International Carry-On): 142만 3,705원
9. 30" Macro Travele